In [1]:
import numpy as np
import pandas as pd

In [3]:
temp_df =pd.read_csv('IMDB Dataset.csv')

In [4]:
df = temp_df.iloc[:10000]

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:

df.drop_duplicates(inplace=True)


C:\Users\parth\AppData\Local\Temp\ipykernel_1664\772323618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [7]:

import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [8]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\parth\AppData\Local\Temp\ipykernel_1664\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [9]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\parth\AppData\Local\Temp\ipykernel_1664\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [10]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\parth\AppData\Local\Temp\ipykernel_1664\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [11]:

df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [12]:
import gensim

In [13]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [14]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [18]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)


In [21]:
model.build_vocab(story)

In [22]:

model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875393, 6212140)

In [23]:

len(model.wv.index_to_key)

31845

In [24]:

def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [25]:
document_vector(df['review'].values[0])


array([-2.02073038e-01,  5.01998127e-01,  1.63309649e-01,  2.52334833e-01,
       -8.02389905e-02, -6.46206141e-01,  2.35104889e-01,  9.39284563e-01,
       -3.31108809e-01, -2.81323820e-01, -3.08281094e-01, -5.27416527e-01,
        5.33612706e-02,  1.32540777e-01,  2.06399322e-01, -7.39664957e-02,
        8.76664557e-03, -3.22644681e-01, -9.31999162e-02, -5.90554535e-01,
        5.69714718e-02,  2.63887823e-01,  8.53212848e-02, -2.53322750e-01,
       -2.78735220e-01,  2.07240781e-04, -3.55418295e-01, -3.80389765e-02,
       -2.86218584e-01,  4.43897992e-02,  3.16602618e-01, -1.69540234e-02,
        1.80413648e-01, -2.32293159e-01, -1.26782402e-01,  4.90005940e-01,
        8.44162703e-02, -3.95458549e-01, -2.65608519e-01, -7.36086369e-01,
        9.56615284e-02, -2.83824682e-01,  6.06130362e-02, -9.59544554e-02,
        5.05553126e-01, -1.13449454e-01, -2.74148256e-01, -7.26595372e-02,
        1.52585909e-01,  3.35546672e-01,  7.07314909e-02, -4.34990585e-01,
       -3.99632424e-01, -

In [26]:
from tqdm import tqdm

In [27]:
x = []
for doc in tqdm(df['review'].values):
    x.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [16:48<00:00,  9.90it/s]


In [28]:
x = np.array(x)

In [29]:
x[0]

array([-2.02073038e-01,  5.01998127e-01,  1.63309649e-01,  2.52334833e-01,
       -8.02389905e-02, -6.46206141e-01,  2.35104889e-01,  9.39284563e-01,
       -3.31108809e-01, -2.81323820e-01, -3.08281094e-01, -5.27416527e-01,
        5.33612706e-02,  1.32540777e-01,  2.06399322e-01, -7.39664957e-02,
        8.76664557e-03, -3.22644681e-01, -9.31999162e-02, -5.90554535e-01,
        5.69714718e-02,  2.63887823e-01,  8.53212848e-02, -2.53322750e-01,
       -2.78735220e-01,  2.07240781e-04, -3.55418295e-01, -3.80389765e-02,
       -2.86218584e-01,  4.43897992e-02,  3.16602618e-01, -1.69540234e-02,
        1.80413648e-01, -2.32293159e-01, -1.26782402e-01,  4.90005940e-01,
        8.44162703e-02, -3.95458549e-01, -2.65608519e-01, -7.36086369e-01,
        9.56615284e-02, -2.83824682e-01,  6.06130362e-02, -9.59544554e-02,
        5.05553126e-01, -1.13449454e-01, -2.74148256e-01, -7.26595372e-02,
        1.52585909e-01,  3.35546672e-01,  7.07314909e-02, -4.34990585e-01,
       -3.99632424e-01, -

In [30]:

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [31]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [33]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [34]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [36]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test,y_pred)


0.7761642463695543